In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pareto
import sys
import pickle
sys.path.append('/home/ziniu.wzn/BayesCard')
from Models.Bayescard_BN import Bayescard_BN
from time import perf_counter
from Evaluation.utils import parse_query
from Evaluation.cardinality_estimation import parse_query_single_table

In [5]:
def discretize_series(s, domain_size):
    n_invalid = len(s[s>=domain_size])
    s = s[s<domain_size]
    s = np.floor(s)
    new_s = np.random.randint(domain_size, size=n_invalid)
    s = np.concatenate((s, new_s))
    return np.random.permutation(s)
    
def data_generation(skew, domain_size, correlation, column_size, nrows=1000000):
    data = np.zeros((column_size, nrows))
    for i in range(column_size):
        if i == 0:
            s = np.random.randint(domain_size, size=nrows)
            data[i,:] = s
            continue
        s = pareto.rvs(b=skew, scale=1, size=nrows)
        s = discretize_series(s, domain_size)
        if i == 1:
            selected_cols = [0]
        else:
            #num_selected_cols = max(np.random.randint(int(np.ceil(i*0.1))), 1)
            num_selected_cols = 1
            selected_cols = np.random.permutation(i)[0:num_selected_cols]
        idx = np.random.permutation(nrows)[0:int(nrows*correlation)]
        if len(idx) != 0:
            selected_data = data[selected_cols, :]
            selected_data = np.ceil(np.mean(selected_data, axis=0))
            s[idx] = selected_data[idx]
        assert len(np.unique(s)) <= domain_size, "invalid domain"
        data[i,:] = s
        
    data = pd.DataFrame(data=data.transpose(), columns=[f"attr{i}" for i in range(column_size)])
    return data

def query_generation(data, table_name, num_sample=200, p=0.8, nval_per_col=4, skip_zero_bit=6):
    queries = []
    cards = []
    for i in range(num_sample):
        query, card = generate_single_query(data, table_name, p, nval_per_col, skip_zero_bit)
        while query is None:
            query, card = generate_single_query(data, table_name, p, nval_per_col, skip_zero_bit)
        queries.append(query)
        cards.append(card)
    return queries, cards

def generate_single_query(df, table_name, p=0.8, nval_per_col=4, skip_zero_bit=6):
    """
    p, nval_per_col, and skip_zero_bit are controlling the true cardinality size. As we know smaller true card 
    generally leads to larger q-error, which will bias the experimental result, so we use this to control the 
    true card to be similar for all experiments.
    """
    query = f"SELECT COUNT(*) FROM {table_name} WHERE "
    execute_query = ""
    column_names = df.columns
    n_cols = 0
    for i, col in enumerate(column_names):
        a = np.random.choice([0,1], p=[p,1-p])
        if a == 0:
            index = np.random.choice(len(df), size=nval_per_col)
            val = sorted(list(df[col].iloc[index]))
            left_val = val[0]
            right_val = val[-1]
            if left_val == right_val:
                sub_query = col + '==' + str(left_val) + ' and '
                act_sub_query = col + ' = ' + str(left_val) + ' AND '
            else:
                if skip_zero_bit:
                    left_val += skip_zero_bit
                    right_val += skip_zero_bit
                sub_query = str(left_val) + ' <= ' + col + ' <= ' + str(right_val) + ' and '
                act_sub_query = col + ' >= ' + str(left_val) + ' AND ' + col + ' <= ' + str(right_val) + ' AND '
            execute_query += sub_query
            query += act_sub_query
    if execute_query == "":
        return None,  None
    execute_query = execute_query[:-5]
    query = query[:-5]
    try:
        card = len(df.query(execute_query))
    except:
        card = 0
    if card==0:
        return None, None
    return query, card

In [6]:
def train_one(skew, domain_size, correlation, column_size, nrows=1000000, num_sample=200, 
              p=0.8, nval_per_col=4, skip_zero_bit=6, rows_to_use=10000, n_mcv=30, n_bins=70):
    data = data_generation(skew, domain_size, correlation, column_size, nrows=nrows)
    name = f"toy_{skew}_{domain_size}_{correlation}_{column_size}"
    queries, cards = query_generation(data, name, num_sample, p, nval_per_col, skip_zero_bit)
    BN = Bayescard_BN(name)
    BN.build_from_data(data, sample_size=rows_to_use, n_mcv=n_mcv, n_bins=n_bins)
    model_path = f'/home/ziniu.wzn/BN_checkpoints/synthetic/{name}.pkl'
    pickle.dump(BN, open(model_path, 'wb'), pickle.HIGHEST_PROTOCOL)
    BN.infer_algo = "exact-jit"
    BN.init_inference_method()
    latencies = []
    q_errors = []
    for query_no, query_str in enumerate(queries):
        query = parse_query_single_table(query_str.strip(), BN)
        cardinality_true = cards[query_no]
        card_start_t = perf_counter()
        cardinality_predict = BN.query(query)
        card_end_t = perf_counter()
        latency_ms = (card_end_t - card_start_t) * 1000
        if cardinality_predict == 0 and cardinality_true == 0:
            q_error = 1.0
        elif np.isnan(cardinality_predict) or cardinality_predict == 0:
            cardinality_predict = 1
            q_error = max(cardinality_predict / cardinality_true, cardinality_true / cardinality_predict)
        elif cardinality_true == 0:
            cardinality_true = 1
            q_error = max(cardinality_predict / cardinality_true, cardinality_true / cardinality_predict)
        else:
            q_error = max(cardinality_predict / cardinality_true, cardinality_true / cardinality_predict)
        latencies.append(latency_ms)
        q_errors.append(q_error)
    for i in [50, 90, 95, 99, 100]:
        print(f"q-error {i}% percentile is {np.percentile(q_errors, i)}")
    print(f"average latency is {np.mean(latencies)} ms")
    return q_errors, latencies

def run_all_experiment():
    parameters_to_explore = {
        "skew": [0.1, 0.3, 0.6, 1.0, 1.5, 2.0],
        "domain_size": [10, 100, 500, 1000, 5000, 10000],
        "correlation": [0, 0.2, 0.4, 0.6, 0.8, 1.0],
        "column_num": [2, 5, 10, 50, 100, 200]
    }
    
    print("runing experiment on varying skewness: [0.1, 0.3, 0.6, 1.0, 1.3, 1.6, 2.0]}")
    print("controlled parameters are: domain_size = 100, correlation = 0.4, column_num = 10")
    skew_qerrors = []
    for skew in parameters_to_explore["skew"]:
        print("============================================================")
        print(f"Tesing skewness = {skew}")
        q_errors, latencies = train_one(skew, 100, 0.4, 10, nrows=1000000)
        skew_qerrors.append(np.asarray(q_errors))
    skew_qerrors = np.stack(skew_qerrors)
    np.save("skew_qerrors", skew_qerrors)
    
    print("============================================================")
    print("============================================================")
    
    print("runing experiment on varying domain_size: [10, 100, 1000, 10000]")
    print("controlled parameters are: skewness = 1.0, correlation = 0.4, column_num = 10")
    domain_size_qerrors = []
    for domain_size in parameters_to_explore["domain_size"]:
        print("============================================================")
        print(f"Tesing domain_size = {domain_size}")
        q_errors, latencies = train_one(1, domain_size, 0.4, 10, nrows=1000000)
        domain_size_qerrors.append(np.asarray(q_errors))
    domain_size_qerrors = np.stack(domain_size_qerrors)
    np.save("domain_size_qerrors", domain_size_qerrors)
    
    print("============================================================")
    print("============================================================")
    
    print("runing experiment on varying correlation: [0, 0.2, 0.4, 0.6, 0.8, 1.0]")
    print("controlled parameters are: skewness = 1.0, domain_size = 100, column_num = 10")
    correlation_qerrors = []
    for correlation in parameters_to_explore["correlation"]:
        print("============================================================")
        print(f"Tesing correlation = {correlation}")
        q_errors, latencies = train_one(1, 100, correlation, 10, nrows=1000000)
        correlation_qerrors.append(np.asarray(q_errors))
    correlation_qerrors = np.stack(correlation_qerrors)
    np.save("correlation_qerrors", correlation_qerrors)
    
    print("============================================================")
    print("============================================================")
    
    print("runing experiment on varying column_num: [2, 5, 10, 50, 100, 200]")
    print("controlled parameters are: skewness = 1.0, domain_size = 100, correlation = 0.4")
    column_num_qerrors = []
    for column_num in parameters_to_explore["column_num"]:
        print("============================================================")
        print(f"Tesing column_num = {column_num}")
        q_errors, latencies = train_one(1, 100, 0.4, column_num, nrows=1000000)
        column_num_qerrors.append(np.asarray(q_errors))
    column_num_qerrors = np.stack(column_num_qerrors)
    np.save("column_num_qerrors", column_num_qerrors)

In [8]:
parameters_to_explore = {
        "skew": ([0.1, 0.3, 0.6, 1.0, 1.5, 2.0], [4,4,3,3,2,2]),
        "domain_size": ([10, 100, 500, 1000, 5000, 10000], [4,4,4,4,4,4]),
        "correlation": ([0, 0.2, 0.4, 0.6, 0.8, 1.0], [4,4,4,4,4,4]),
        "column_num": ([2, 5, 10, 50, 100, 200], [6,6,4,2,0,0])
    }

In [7]:

skew = parameters_to_explore["skew"][0]
print(f"runing experiment on varying skewness: {skew}")
print("controlled parameters are: domain_size = 100, correlation = 0.4, column_num = 10")
skew_qerrors = []
for i in range(len(skew)):
    skew = parameters_to_explore["skew"][0][i]
    skip_zero_bit = parameters_to_explore["skew"][1][i]
    print("============================================================")
    print(f"Tesing skewness = {skew}")
    q_errors, latencies = train_one(skew, 100, 0.4, 10, skip_zero_bit=skip_zero_bit)
    skew_qerrors.append(np.asarray(q_errors))
skew_qerrors = np.stack(skew_qerrors)
np.save("skew_qerrors", skew_qerrors)

runing experiment on varying skewness: [0.1, 0.3, 0.6, 1.0, 1.3, 1.6, 2.0]}
controlled parameters are: domain_size = 100, correlation = 0.4, column_num = 10
Tesing skewness = 0.1
Discretizing table takes 7.742928981781006 secs
Structure learning took 1.8320200443267822 secs.
done, parameter learning took 9.03601861000061 secs.
q-error 50% percentile is 1.019442784641686
q-error 90% percentile is 1.0479473167136624
q-error 95% percentile is 1.0613888315042053
q-error 99% percentile is 1.0814405254338593
q-error 100% percentile is 1.1321119017840218
average latency is 7.15815756469965 ms
Tesing skewness = 0.3
Discretizing table takes 6.3387229442596436 secs
Structure learning took 1.875269889831543 secs.
done, parameter learning took 10.549920320510864 secs.
q-error 50% percentile is 1.0236816349813394
q-error 90% percentile is 1.0696273039901292
q-error 95% percentile is 1.0934889963221888
q-error 99% percentile is 1.3826168392239966
q-error 100% percentile is 1.8911336577282893
average

In [11]:
domain_size = parameters_to_explore["domain_size"][0]
print(f"runing experiment on varying domain_size: {domain_size}")
print("controlled parameters are: skewness = 1.0, correlation = 0.4, column_num = 10")
domain_size_qerrors = []
for i in range(len(domain_size)):
    domain_size = parameters_to_explore["domain_size"][0][i]
    skip_zero_bit = parameters_to_explore["domain_size"][1][i]
    print("============================================================")
    print(f"Tesing domain_size = {domain_size}")
    q_errors, latencies = train_one(1, domain_size, 0.4, 10, nrows=1000000, skip_zero_bit=skip_zero_bit)
    domain_size_qerrors.append(np.asarray(q_errors))
domain_size_qerrors = np.stack(domain_size_qerrors)
np.save("domain_size_qerrors", domain_size_qerrors)

runing experiment on varying domain_size: [10, 100, 500, 1000, 5000, 10000]
controlled parameters are: skewness = 1.0, correlation = 0.4, column_num = 10
Tesing domain_size = 10
Discretizing table takes 3.2726800441741943 secs
Structure learning took 1.0676326751708984 secs.
done, parameter learning took 6.7733728885650635 secs.
q-error 50% percentile is 1.0283518911708862
q-error 90% percentile is 1.1685129334542326
q-error 95% percentile is 1.2930413036356396
q-error 99% percentile is 1.712292196714191
q-error 100% percentile is 4.76605852747474
average latency is 2.072511501610279 ms
Tesing domain_size = 100
Discretizing table takes 5.959499359130859 secs
Structure learning took 1.753504991531372 secs.
done, parameter learning took 8.807895421981812 secs.
q-error 50% percentile is 1.0837036888106923
q-error 90% percentile is 1.4317781550184194
q-error 95% percentile is 1.5708856085079208
q-error 99% percentile is 2.769807338265714
q-error 100% percentile is 8.460473231771633
average

In [21]:
correlation = parameters_to_explore["correlation"][0]
print(f"runing experiment on varying correlation: {correlation}")
print("controlled parameters are: skewness = 1.0, domain_size = 100, column_num = 10")
correlation_qerrors = []
for i in range(len(correlation)):
    domain_size = parameters_to_explore["correlation"][0][i]
    skip_zero_bit = parameters_to_explore["correlation"][1][i]
    print("============================================================")
    print(f"Tesing correlation = {correlation}")
    q_errors, latencies = train_one(1, 100, correlation, 10, nrows=1000000, skip_zero_bit=skip_zero_bit)
    correlation_qerrors.append(np.asarray(q_errors))
correlation_qerrors = np.stack(correlation_qerrors)
np.save("correlation_qerrors", correlation_qerrors)

runing experiment on varying correlation: [0, 0.2, 0.4, 0.6, 0.8, 1.0]
controlled parameters are: skewness = 1.0, domain_size = 100, column_num = 10
Tesing correlation = 0
Discretizing table takes 6.252589225769043 secs
Structure learning took 105.05017137527466 secs.
done, parameter learning took 7.225903749465942 secs.
q-error 50% percentile is 1.5520722744737596
q-error 90% percentile is 9.678739917888748
q-error 95% percentile is 27.104876262849558
q-error 99% percentile is 121.51581935214253
q-error 100% percentile is 1030.289481648038
average latency is 4.49909683316946 ms
Tesing correlation = 0.2
Discretizing table takes 6.2613749504089355 secs
Structure learning took 103.8951849937439 secs.
done, parameter learning took 7.121143341064453 secs.
q-error 50% percentile is 1.427032086866471
q-error 90% percentile is 3.5458052818169
q-error 95% percentile is 7.945919027161713
q-error 99% percentile is 13.551710462032924
q-error 100% percentile is 30.27929504373782
average latency is

In [13]:
column_num = parameters_to_explore["column_num"][0]
print(f"runing experiment on varying column_num: {column_num}")
print("controlled parameters are: skewness = 1.0, domain_size = 100, correlation = 0.4")
column_num_qerrors = []
p_list = [0.8, 0.8, 0.8, 0.2, 0.1, 0.05]
for i in range(len(column_num)):
    column_num = parameters_to_explore["column_num"][0][i]
    skip_zero_bit = parameters_to_explore["column_num"][1][i]
    print("============================================================")
    print(f"Tesing column number = {column_num}")
    p = p_list[i]
    q_errors, latencies = train_one(1, 100, 0.4, column_num, p=p, skip_zero_bit=skip_zero_bit, nrows=1000000)
    column_num_qerrors.append(np.asarray(q_errors))
column_num_qerrors = np.stack(column_num_qerrors)
np.save("column_num_qerrors", column_num_qerrors)

runing experiment on varying column_num: [2, 5, 10, 50, 100, 200]
controlled parameters are: skewness = 1.0, domain_size = 100, correlation = 0.4
Tesing column number = 2
Discretizing table takes 1.2706284523010254 secs
Structure learning took 0.15340089797973633 secs.
done, parameter learning took 0.4135129451751709 secs.


/home/ziniu.wzn/anaconda3/envs/deepdb/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
/home/ziniu.wzn/anaconda3/envs/deepdb/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


q-error 50% percentile is 1.0005554448935015
q-error 90% percentile is 1.0128038297429311
q-error 95% percentile is 1.0185697335971853
q-error 99% percentile is 1.05202542813193
q-error 100% percentile is 1.2479818364055857
average latency is 1.2471112236380577 ms
Tesing column number = 5
Discretizing table takes 3.1535093784332275 secs
Structure learning took 0.7785553932189941 secs.
done, parameter learning took 0.7976157665252686 secs.
q-error 50% percentile is 1.0216399616086824
q-error 90% percentile is 1.067617156976904
q-error 95% percentile is 1.1193217617163849
q-error 99% percentile is 1.1963113954565479
q-error 100% percentile is 1.330873324450124
average latency is 4.372875913977623 ms
Tesing column number = 10
Discretizing table takes 6.959289312362671 secs
Structure learning took 1.713876485824585 secs.
done, parameter learning took 3.618835210800171 secs.
q-error 50% percentile is 1.064602911691702
q-error 90% percentile is 1.3131823292994318
q-error 95% percentile is 1.